In [16]:
rc_2007_01 = spark.read.format('json').load('hdfs://orion11:15000/rc/2007/RC_2007-01.bz2')
rc_2007_01.cache()

DataFrame[author: string, author_flair_css_class: string, author_flair_text: string, body: string, controversiality: bigint, created_utc: bigint, distinguished: string, edited: string, gilded: bigint, id: string, link_id: string, parent_id: string, retrieved_on: bigint, score: bigint, stickied: boolean, subreddit: string, subreddit_id: string, ups: bigint]

In [21]:
rc = spark.read.format('json').load('hdfs://orion11:15000/rc/*')

In [25]:
rc

DataFrame[archived: boolean, author: string, author_flair_css_class: string, author_flair_text: string, body: string, controversiality: bigint, created_utc: string, distinguished: string, downs: bigint, edited: string, gilded: bigint, id: string, link_id: string, name: string, parent_id: string, removal_reason: string, retrieved_on: bigint, score: bigint, score_hidden: boolean, stickied: boolean, subreddit: string, subreddit_id: string, ups: bigint]

In [120]:
rc_samp = rc.sample(False, .1)

In [121]:
rc_samp.cache()

DataFrame[archived: boolean, author: string, author_flair_css_class: string, author_flair_text: string, body: string, controversiality: bigint, created_utc: string, distinguished: string, downs: bigint, edited: string, gilded: bigint, id: string, link_id: string, name: string, parent_id: string, removal_reason: string, retrieved_on: bigint, score: bigint, score_hidden: boolean, stickied: boolean, subreddit: string, subreddit_id: string, ups: bigint]

In [125]:
rc_samp.write.format('json').save('hdfs://orion11:15000/rc_samp')

In [9]:
type(rc_2007_01)
rc_2007_01.columns

['author',
 'author_flair_css_class',
 'author_flair_text',
 'body',
 'controversiality',
 'created_utc',
 'distinguished',
 'edited',
 'gilded',
 'id',
 'link_id',
 'parent_id',
 'retrieved_on',
 'score',
 'stickied',
 'subreddit',
 'subreddit_id',
 'ups']

In [7]:
rc_2007_01.createOrReplaceTempView("rc_2007_01")

In [17]:
rc_2007_01.rdd \
    .map(lambda comment: ('key', 1)) \
    .reduceByKey(lambda accum, n: accum + n) \
    .collect()

[('key', 81341)]

In [22]:
spark.sql("\
SELECT COUNT(*) \
FROM rc_2007_01").show()

+--------+
|count(1)|
+--------+
|   81341|
+--------+



In [30]:
sub_count_mr = rc_2007_01.rdd \
    .map(lambda comment: (comment['subreddit'], 1)) \
    .reduceByKey(lambda accum, n: accum + n) \
    .collect()

In [147]:
print(len(sub_count_mr), sub_count_mr)

23 [('programming', 7650), ('reddit.com', 70939), ('science', 2018), ('tr', 30), ('sl', 24), ('lipstick.com', 98), ('pl', 7), ('fr', 11), ('ja', 138), ('freeculture', 101), ('joel', 54), ('request', 36), ('arxiv', 4), ('features', 156), ('nl', 2), ('de', 34), ('nsfw', 7), ('it', 11), ('es', 2), ('zh', 1), ('rlc', 1), ('ru', 15), ('sv', 2)]


In [34]:
len(sub_count_mr)

23

In [47]:
sub_count_sql = spark.sql("\
SELECT subreddit, COUNT(*) as cnt \
FROM rc_2007_01 \
GROUP BY subreddit").collect()

In [48]:
sub_count_sql

[Row(subreddit='sl', cnt=24),
 Row(subreddit='pl', cnt=7),
 Row(subreddit='arxiv', cnt=4),
 Row(subreddit='programming', cnt=7650),
 Row(subreddit='lipstick.com', cnt=98),
 Row(subreddit='reddit.com', cnt=70939),
 Row(subreddit='tr', cnt=30),
 Row(subreddit='de', cnt=34),
 Row(subreddit='es', cnt=2),
 Row(subreddit='nsfw', cnt=7),
 Row(subreddit='request', cnt=36),
 Row(subreddit='science', cnt=2018),
 Row(subreddit='it', cnt=11),
 Row(subreddit='sv', cnt=2),
 Row(subreddit='nl', cnt=2),
 Row(subreddit='freeculture', cnt=101),
 Row(subreddit='ru', cnt=15),
 Row(subreddit='features', cnt=156),
 Row(subreddit='joel', cnt=54),
 Row(subreddit='zh', cnt=1),
 Row(subreddit='fr', cnt=11),
 Row(subreddit='ja', cnt=138),
 Row(subreddit='rlc', cnt=1)]

In [46]:
spark.sql("\
SELECT COUNT(DISTINCT subreddit) \
FROM rc_2007_01").show()

+-------------------------+
|count(DISTINCT subreddit)|
+-------------------------+
|                       23|
+-------------------------+



In [133]:
%%time

import string

def screamer_sub_mapper(comment):
    n_upper = len(list(filter(lambda c: c in string.ascii_uppercase, comment['body'])))
    n_alpha = len(list(filter(lambda c: c in string.ascii_letters, comment['body'])))
    return (comment['subreddit'], (n_upper, n_alpha))

def screamer_reducer(value_list):
    total_upper = 0
    total_alpha = 0
    for value in value_list:
        (n_upper, n_alpha) = value
        total_upper += n_upper
        total_alpha += n_alpha
    screamer_score = total_upper / total_alpha if total_alpha else 0
    return (screamer_score, len(value_list))
    
screamer_subs = rc_samp.rdd \
    .map(screamer_sub_mapper) \
    .groupByKey() \
    .mapValues(screamer_reducer) \

screamer_subs.cache()

CPU times: user 50.4 ms, sys: 9.97 ms, total: 60.3 ms
Wall time: 59.1 s


In [144]:
screamer_subs \
    .filter(lambda key0_rval1: key0_rval1[1][1] > 1000) \
    .sortBy(lambda key0_rval1: key0_rval1[1][0], False) \
    .take(3)

[('spacedicks', (0.5781305686052163, 1385)),
 ('circlejerk', (0.18934631577970992, 45861)),
 ('googleplusinvites', (0.11052298941380945, 1089))]